In [283]:
import pandas as pd 
import numpy as np
import re
from datetime import datetime
import ast 

In [716]:
recipes_df = pd.read_csv('data/recipes60k.csv')

In [717]:
pd.set_option('display.max_columns', None)

In [718]:
recipes_df.head(2)

,Unnamed: 0,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",3.0,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."


### Columns to Drop 

In [719]:
recipes_df = recipes_df.drop(columns = ['AuthorId',
                                  'AuthorName',
                                  'DatePublished',
                                   'Description',
                                  'Images',
                                       'Unnamed: 0'])

In [720]:
recipes_df = recipes_df.drop(columns = ['RecipeYield'])

### Drop columns with missing values

In [721]:
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    60000 non-null  int64  
 1   Name                        60000 non-null  object 
 2   CookTime                    48701 non-null  object 
 3   PrepTime                    60000 non-null  object 
 4   TotalTime                   60000 non-null  object 
 5   RecipeCategory              59999 non-null  object 
 6   Keywords                    59428 non-null  object 
 7   RecipeIngredientQuantities  59998 non-null  object 
 8   RecipeIngredientParts       60000 non-null  object 
 9   AggregatedRating            46378 non-null  float64
 10  ReviewCount                 47037 non-null  float64
 11  Calories                    60000 non-null  float64
 12  FatContent                  60000 non-null  float64
 13  SaturatedFatContent         600

In [722]:
recipe_clean_df = recipes_df.dropna(how='any',axis=0) 

In [723]:
recipe_clean_df = recipe_clean_df.reset_index()

In [724]:
#recipe_clean_df.to_csv('data/clean_recipes.csv')

## Prep Time and Cook Time into floats

In [725]:
# remove the string character 
recipe_clean_df['CookTime'] = recipe_clean_df['CookTime'].str.replace(r'PT', '')
recipe_clean_df['PrepTime'] = recipe_clean_df['PrepTime'].str.replace(r'PT', '')
recipe_clean_df['TotalTime'] = recipe_clean_df['TotalTime'].str.replace(r'PT', '')

In [726]:
# Convert to date time
recipe_clean_df['CookTime'] = recipe_clean_df['CookTime'].str.replace(r'M', '')
recipe_clean_df['PrepTime'] = recipe_clean_df['PrepTime'].str.replace(r'M', '')
recipe_clean_df['TotalTime'] = recipe_clean_df['TotalTime'].str.replace(r'M', '')

In [727]:
# # convert the hours in to minutes for Cooktime column
# for count, value in enumerate(recipe_clean_df['CookTime']):
#     if type(recipe_clean_df['CookTime'][count]) == int:
#         break
#     elif 'H' in recipe_clean_df['CookTime'][count]:
#         recipe_clean_df['CookTime'][count] = (int(recipe_clean_df['CookTime'][0][:-1])) * 60

In [728]:
# convert the hours in to minutes for PrepTime column
# for count, value in enumerate(recipe_clean_df['PrepTime']):
#     if 'H' in recipe_clean_df['PrepTime'][count]:
#         recipe_clean_df['PrepTime'][count] = (int(recipe_clean_df['PrepTime'][0][:-1])) * 60

### Function to convert string time to minutes

In [729]:
def time_converter(column):
    for count, value in enumerate(recipe_clean_df[f'{column}']):
        if type(value) == str:
            for count2, item in enumerate(value):
                if item == 'H':
                    if value[-1:]  in 'H':
                        recipe_clean_df[f'{column}'][count] = float(recipe_clean_df[f'{column}'][count][:value.index(item)]) * 60
                    else:
                        recipe_clean_df[f'{column}'][count] = float(recipe_clean_df[f'{column}'][count][:value.index(item)]) * 60 + (float(recipe_clean_df[f'{column}'][count][value.index(item)+1:]))
                        
                elif item == 'S':
                    recipe_clean_df[f'{column}'][count] = 0

In [730]:
time_converter('TotalTime')
time_converter('PrepTime')
time_converter('CookTime')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i

In [731]:
recipe_clean_df['TotalTime'] = recipe_clean_df['TotalTime'].astype(str).astype(float)
recipe_clean_df['PrepTime'] = recipe_clean_df['PrepTime'].astype(str).astype(float)
recipe_clean_df['CookTime'] = recipe_clean_df['CookTime'].astype(str).astype(float)

In [732]:
recipe_clean_df[recipe_clean_df['TotalTime'].apply(lambda x: isinstance(x, float))]

,index,RecipeId,Name,CookTime,PrepTime,TotalTime,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions
0,0,38,Low-Fat Berry Blue Frozen Dessert,1440.0,45.0,1485.0,Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,1,39,Biryani,25.0,240.0,265.0,Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",3.0,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,"c(""Soak saffron in warm milk for 5 minutes and..."
2,2,40,Best Lemonade,5.0,30.0,35.0,Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,3,41,Carina's Tofu-Vegetable Kebabs,20.0,1440.0,1460.0,Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""12"", ""1"", ""2"", ""1"", ""10"", ""1"", ""3"", ""2"", ""2...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",4.5,2.0,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,"c(""Drain the tofu, carefully squeezing out exc..."
4,4,42,Cabbage Soup,30.0,20.0,50.0,Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,"c(""Mix everything together and bring to a boil..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22461,59988,64339,Rellenos Jaibas,3.0,15.0,18.0,Crab,"c(""Mexican"", ""Spicy"", ""< 30 Mins"", ""Stove Top"")","c(""1/2"", ""4"", ""1"", ""1/4"", ""4"", ""2"", ""1/4"", ""3""...","c(""asparagus spears"", ""fresh tomato"", ""ricotta...",5.0,4.0,416.6,29.7,7.0,227.2,115.8,25.4,3.1,7.1,14.4,2.0,"c(""Remove stems from chilies, make a long slit..."
22462,59989,64340,Great Canadian Butter Tarts,25.0,30.0,55.0,Dessert,"c(""Canadian"", ""< 60 Mins"", ""For Large Groups"",...","c(""16"", ""2"", ""1"", ""1"", ""1/4"", ""1"", ""1"", ""2"")","c(""eggs"", ""seedless raisin"", ""brown sugar"", ""m...",5.0,8.0,116.1,3.7,0.8,27.1,48.6,20.7,0.3,18.7,1.1,16.0,"c(""Preheat oven to 350 degrees F."", ""Place she..."
22463,59990,64341,Party Time Bruschetta,6.0,20.0,26.0,Vegetable,"c(""Canadian"", ""Broil/Grill"", ""< 30 Mins"", ""Oven"")","c(""4"", ""1"", ""2"", ""1/4"", ""1/4"", ""1"", NA, NA, NA)","c(""fresh tomato"", ""fresh basil"", ""salt"", ""fres...",5.0,1.0,25.5,0.3,0.1,0.0,106.0,4.9,0.4,0.8,0.8,12.0,"c(""Preheat BBQ to medium."", ""Grill 4 slices of..."
22464,59991,64342,Portabella Mushroom Burgers,10.0,10.0,20.0,Vegetable,"c(""Canadian"", ""Summer"", ""Broil/Grill"", ""< 30 M...","c(""4"", NA, NA, NA, ""1"", NA, NA, NA)","c(""portabella mushrooms"", ""asiago cheese"", ""pe...",5.0,1.0,21.8,0.2,0.0,0.0,5.0,4.3,1.3,1.5,2.1,4.0,"c(""Preheat BBQ to medium-high."", ""Brush mushro..."


In [733]:
# recipe_df_1 = recipe_clean_df.copy()

In [734]:
# recipe_clean_df.to_csv('./data/recipes141.csv', sep='\t', encoding='utf-8')

In [735]:
recipes10_df = recipe_clean_df[:10_000]

In [736]:
# recipes10_df = pd.read_csv('data/recipes10.csv', error_bad_lines=False, encoding='utf-8')

In [737]:
# recipes10k.to_csv('./data/recipes10.csv', sep=',', encoding='utf-8')

In [738]:
recipes10_df = recipes10_df.loc[~recipes10_df.RecipeIngredientParts.str.contains("character",  na=False)].reset_index()

### Change string to a list

In [739]:
def list_convert(dataframe, column):
    for count, value in enumerate(dataframe[f'{column}']): 
#         if dataframe[f'{column}'].str.contains('char'):
#             dataframe[f'{column}'][count] = "NA"
        
        
        dataframe[f'{column}'][count] = dataframe[f'{column}'][count].replace('c(', '')
        dataframe[f'{column}'][count] = dataframe[f'{column}'][count].replace(')', '')
        dataframe[f'{column}'][count] = dataframe[f'{column}'][count].replace('0', '')
        dataframe[f'{column}'][count] = dataframe[f'{column}'][count].replace('(', '')
        dataframe[f'{column}'][count] = dataframe[f'{column}'][count].replace('\n', '')
#         dataframe[f'{column}'][count] = dataframe[f'{column}'][count].replace('character', '')
        
        dataframe[f'{column}'][count] = ast.literal_eval(str(dataframe[f'{column}'][count]))
    
#         dataframe[f'{column}'][count] = dataframe[f'{column}'][count].split(",")
#         dataframe[f'{column}'][count] = dataframe[f'{column}'][count].spl
#     recipe_clean_df[f'{column}'] = recipe_clean_df[f'{column}'].str.replace('"', '')
    
#     dataframe[f'{column}'] = dataframe[f'{column}'].apply(lambda x: list(dataframe[f'{column}']))
    

In [801]:
list_convert(recipes10_df, 'Keywords')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/

In [800]:
list_convert(recipes10_df, 'RecipeIngredientQuantities')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/

ValueError: malformed node or string: <_ast.Name object at 0x7fc7d1747f90>

In [740]:
list_convert(recipes10_df, 'RecipeIngredientParts')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/

In [797]:
list_convert(recipes10_df, 'RecipeInstructions')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/

In [798]:
recipes10_df['RecipeInstructions']

0       (Into a 1 quart Jar with tight fitting lid, pu...
1       (Mix everything together and bring to a boil.,...
2       (Melt 1 1/2 ozs butter, add the flour and cook...
3       (Preheat oven to 35°F., Make pie crust, using ...
4       (Cook mushrooms in 2 tbsp butter in a large  s...
                              ...                        
5560    (Heat two tablespoons vegetable oil in a non-s...
5561    (Bring 6 cups water to boil., Add sliced carro...
5562    (In oven proof large pan, brown beef, onions,a...
5563    (Preheat oven to 35 degrees F., In al large sa...
5564    (Melt butter in sauce pan., Add lemon juice, W...
Name: RecipeInstructions, Length: 5565, dtype: object

In [ ]:
# recipes10_df = recipes10_df.drop(columns = ['Unnamed: 0', 'index', 'Unnamed: 0.1'])

In [118]:
# recipes10_df.to_csv('./data/recipes10k_df.csv', sep=',', encoding='utf-8')

#### No cook time over four hours

In [741]:
recipes10_df = recipes10_df[(recipes10_df['CookTime'] <= 240) & (recipes10_df['TotalTime'] <= 480)]

#### Check Data is in correct range 

In [742]:
recipes10_df = recipes10_df[(recipes10_df['RecipeServings'] > 0) & (recipes10_df['RecipeServings'] < 20)]

#### Return recipes where Number of reviews is greater than 2

In [743]:
recipes10_df = recipes10_df[recipes10_df['ReviewCount'] > 2]

#### Create calories per serving column 

In [744]:
recipes10_df['calories_per_serving'] = recipes10_df['Calories']/recipes10_df['RecipeServings']

In [746]:
# recipes10_df = recipes10_df[recipes10_df['calories_per_serving'] < 500].reset_index().drop(columns = 'index')

In [747]:
recipes10_df = recipes10_df.drop(columns = ['level_0', 'RecipeId'])

In [754]:
recipes10_df = recipes10_df.reset_index()

In [813]:
def tuple_to_list(dataframe, column):
    for count, value in enumerate(dataframe[f'{column}']):
        dataframe[f'{column}'][count] = list(dataframe[f'{column}'][count])

In [814]:
tuple_to_list(recipes10_df, 'RecipeIngredientParts')
tuple_to_list(recipes10_df, 'Keywords')
tuple_to_list(recipes10_df, 'RecipeInstructions')

/Users/SeanMcNamara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [809]:
# recipes10_df['RecipeIngredientParts'] = list(recipes10_df['RecipeIngredientParts'])
# recipes10_df['Keywords'] = list(recipes10_df['Keywords'])
# recipes10_df['RecipeInstructions'] = list(recipes10_df['RecipeInstructions'])

In [822]:
# recipes10_df.to_csv('./data/recipes_clean_10k_df', index = False)

In [792]:
import json

In [825]:
recipes10_df.to_pickle('./data/recipes_clean_10k')

In [824]:
type(recipes10_df['RecipeIngredientParts'][1])

list

In [836]:
recipes10_df['RecipeIngredientQuantities'][2]

'c("12", "2", "3", "450", "1", "2", "1/4", "1", NA, NA, "2", "2", "1", NA)'